In [3]:
import json
import requests

class ChirpExtraction():

    def __init__(self, email, password, graphql_endpoint="https://api.chirpsystems.com/graphql", august_endpoint="https://api-production.august.com"):
        self.graphql_endpoint = graphql_endpoint
        self.august_endpoint = august_endpoint

        self.graphql_calls = {
        
            "determine_user_auth": {
                "json": {
                    "operationName": "determineAuthMethod",
                    "variables": {
                        "input": {
                        "userIdentifier": {
                            "type": "EMAIL",
                            "value": email
                        }
                     }
                    },
                    "query": "query determineAuthMethod($input: DetermineAuthMethodInput!) {\n  determineAuthMethod(input: $input)\n}"
                }
            },
            "post_username_password": {
                "json": {
                  "operationName": "verifyPassword",
                  "variables": {
                    "input": {
                      "userIdentifier": {
                        "type": "EMAIL",
                        "value": email
                      },
                      "password": password,
                      "sessionType": "ACCESS_APP"
                    }
                  },
                  "query": "mutation verifyPassword($input: VerifyPasswordInput!) {\n  verifyPassword(input: $input) {\n    verificationToken\n    __typename\n  }\n}"
                }
            },
            "send_mfa_password": {
                "json": {
                    "operationName": "sendLoginCode",
                    "variables": {
                      "input": {
                        "userIdentifier": {
                          "type": "EMAIL",
                          "value": email
                        },
                        "sessionType": "ACCESS_APP",
                        "verificationToken": None
                      }
                    },
                    "query": "mutation sendLoginCode($input: SendLoginCodeInput!) {\n  verificationToken: sendLoginCode(input: $input)\n}"
                }
            },
            "finalize_login": {
                "json": {
                  "operationName": "verifyLoginCode",
                  "variables": {
                    "input": {
                      "identifier": email,
                      "code": None,
                      "verificationToken": None,
                      "firstName": "",
                      "lastName": ""
                    }
                  },
                  "query": "mutation verifyLoginCode($input: VerifyLoginCodeInput!) {\n  verifyLoginCode(input: $input) {\n    verificationToken\n    session {\n      ...SessionFragment\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment SessionFragment on SessionOutput {\n  augustAccessToken\n  chirpAccessToken\n  chirpRefreshToken\n  expiresAt\n  chirpUser {\n    userId\n    augustUserId: augustSubUserId\n    __typename\n  }\n  __typename\n}"
                }
            },
            "get_august_info": {
                "json": {
                    "operationName": "currentUser",
                    "variables": {
                      "operatingSystem": "ANDROID"
                    },
                    "query": "query currentUser($operatingSystem: OperatingSystem!) {\n  latestAccessAppVersion(operatingSystem: $operatingSystem)\n  currentUser {\n    ...UserFragment\n    hasPassword\n    isMockUser\n    lastReadAnnouncementsAt\n    acceptedEulaAt\n    assignedPinCodes {\n      pinCodeId\n      code\n      propertyId\n      __typename\n    }\n    mobileCredentials {\n      identifier\n      __typename\n    }\n    userRoles {\n      userRoleId\n      type\n      scopedPropertyId\n      __typename\n    }\n    announcements {\n      ...AnnouncementFragment\n      __typename\n    }\n    assignedAccessKeys {\n      ...AccessKeyFragment\n      __typename\n    }\n    guestsAccessKeys {\n      ...GuestAccessKeyFragment\n      __typename\n    }\n    appDisplay {\n      ingressOnly\n      totalPropertyAccess\n      __typename\n    }\n    favoritesByUserId {\n      accessPointSettingsId\n      accessPointId\n      position\n      favorited\n      __typename\n    }\n    __typename\n  }\n  accessPoints {\n    ...AccessPointFragment\n    __typename\n  }\n  accessAppConfig {\n    AUGUST_API_KEY\n    __typename\n  }\n}\n\nfragment UserFragment on User {\n  userId\n  userUuid\n  firstName\n  lastName\n  phoneNumber\n  email\n  augustUserId: augustSubUserId\n  virtualDirectoryEnabled\n  __typename\n}\n\nfragment AnnouncementFragment on Announcement {\n  announcementId\n  title\n  message\n  createdAt\n  preview\n  __typename\n}\n\nfragment AccessKeyFragment on AccessKey {\n  accessKeyId\n  accessKeyUuid\n  accessFrequency\n  activationStatus\n  activatesAt\n  expiresAt\n  nickname\n  type\n  timezone\n  revokedAt\n  propertyId\n  notes\n  reason\n  reasonEnum {\n    label\n    value\n    __typename\n  }\n  property {\n    ...PropertyFragment\n    __typename\n  }\n  appDisplay {\n    rowHeading1\n    rowHeading2\n    __typename\n  }\n  unit {\n    unitId\n    type\n    displayName\n    propertyId\n    property {\n      ...PropertyFragment\n      __typename\n    }\n    smartLocks {\n      ...SmartLockFragment\n      __typename\n    }\n    __typename\n  }\n  timetable {\n    ...TimetableRuleFragment\n    __typename\n  }\n  assignedToUser {\n    ...UserFragment\n    userRoles {\n      userRoleId\n      type\n      scopedPropertyId\n      __typename\n    }\n    __typename\n  }\n  accessPointEventsByAccessKeyId {\n    ...AccessPointEventFragment\n    __typename\n  }\n  __typename\n}\n\nfragment PropertyFragment on Property {\n  propertyId\n  name\n  imageId\n  featureFlags\n  supportPhoneNumber\n  image {\n    imageId\n    cloudId\n    __typename\n  }\n  __typename\n}\n\nfragment SmartLockFragment on SmartLock {\n  smartLockId\n  unitId\n  name\n  sourceId\n  enabled\n  serialNumber\n  __typename\n}\n\nfragment TimetableRuleFragment on TimetableRule {\n  days\n  start\n  end\n  type\n  state\n  startDate\n  endDate\n  __typename\n}\n\nfragment AccessPointEventFragment on AccessPointEvent {\n  accessPointEventId\n  accessPointId\n  createdByUserId\n  endsAt\n  removedAt\n  startsAt\n  state\n  __typename\n}\n\nfragment GuestAccessKeyFragment on AccessKey {\n  accessKeyId\n  accessFrequency\n  accessKeyUuid\n  activationStatus\n  activatesAt\n  expiresAt\n  nickname\n  type\n  timezone\n  revokedAt\n  notes\n  reason\n  propertyId\n  unit {\n    unitId\n    displayName\n    propertyId\n    __typename\n  }\n  timetable {\n    ...TimetableRuleFragment\n    __typename\n  }\n  assignedToUser {\n    ...UserFragment\n    __typename\n  }\n  __typename\n}\n\nfragment AccessPointFragment on AccessPoint {\n  accessPointId\n  friendlyName\n  accessTimeout\n  timezone\n  reservationOnly\n  type\n  maxOccupancy\n  imageId\n  proximityRestriction\n  spaceshipModeEnabled\n  staffAccessOnly\n  image {\n    imageId\n    cloudId\n    __typename\n  }\n  property {\n    name\n    propertyId\n    image {\n      imageId\n      cloudId\n      __typename\n    }\n    __typename\n  }\n  timetable {\n    ...TimetableRuleFragment\n    __typename\n  }\n  beacons {\n    ...BeaconFragment\n    macAddress\n    __typename\n  }\n  readers {\n    readerId\n    address\n    serialNumber\n    childHubId\n    __typename\n  }\n  __typename\n}\n\nfragment BeaconFragment on Beacon {\n  beaconId\n  accessPointId\n  name\n  uuid\n  major\n  minor\n  __typename\n}"
                }
            }
        }

        self.headers = {
            "User-Agent" : "okhttp/4.12.0",
            "x-device-fingerprint": "native:57ab20ef12",
            "x-device-info": '{"appVersion":"1.33.0","brand":"google","model":"sdk_gphone64_x86_64","system":"Android","systemVersion":"14","homeAccessAuth":false}',
            "Host": "api.chirpsystems.com",
            "accept": "*/*",
            "Connection": "Keep-Alive",
            "Accept-Encoding": "gzip",
            "Content-Type": "application/json"
        }

    def get_auth_type(self):
        req = requests.post(self.graphql_endpoint, json=self.graphql_calls["determine_user_auth"]["json"], headers=self.headers)
        #print("get auth type", req.json())
        return req.json()

    def get_verification_token(self):
        req = requests.post(self.graphql_endpoint, json=self.graphql_calls["post_username_password"]["json"], headers=self.headers)
        #print("get verification token", req.json())
        return req.json()["data"]["verifyPassword"]["verificationToken"]
    
    def send_mfa_password(self, verification_token):
        self.graphql_calls["send_mfa_password"]["json"]["variables"]["input"]["verificationToken"] = verification_token
        req = requests.post(self.graphql_endpoint, json=self.graphql_calls["send_mfa_password"]["json"], headers=self.headers)
        #print("get mfa password", req.json())
        return req.json()["data"]["verificationToken"]
    
    def finalize_auth(self, mfa_verification_token, email_code):
        self.graphql_calls["finalize_login"]["json"]["variables"]["input"]["verificationToken"] = mfa_verification_token
        self.graphql_calls["finalize_login"]["json"]["variables"]["input"]["code"] = email_code
        req = requests.post(self.graphql_endpoint, json=self.graphql_calls["finalize_login"]["json"], headers=self.headers)
        return req.json()
    
    def get_august_critical_info(self, chirp_access_token, august_access_token):
        curr_headers = {
            "authorization": chirp_access_token,
            "x-august-access-token": august_access_token
        }
        req = requests.post(self.graphql_endpoint, json=self.graphql_calls["get_august_info"]["json"], headers=curr_headers)
        data_req = req.json()
        return {
            "augustApiKey": data_req["data"]["accessAppConfig"]["AUGUST_API_KEY"],
            "lockId": data_req["data"]["currentUser"]["assignedAccessKeys"][0]["unit"]["smartLocks"][0]["sourceId"],
            "augustAccessToken": august_access_token
        }
    
    def get_august_details(self, august_access_token, august_api_key, lock_id):
        curr_headers = {
            "x-august-api-key": august_api_key,
            "x-august-access-token": august_access_token,
            "Content-Type": "application/json"
        }
        req = requests.get(f"{self.august_endpoint}/locks/{lock_id}", headers=curr_headers)
        data_req = req.json()

        return {
            "secretKey": data_req["OfflineKeys"]["loaded"][0]["key"],
            "macAddress": data_req["macAddress"],
            "serialNumber": data_req["SerialNumber"],
            "slotNumber": data_req["OfflineKeys"]["loaded"][0]["slot"],
        }

with open('config.json', 'r') as fp:
    curr_config = json.load(fp)

    ce = ChirpExtraction(curr_config["username"], curr_config["password"])
    ce.get_auth_type()
    vt = ce.get_verification_token()

    vtt = ce.send_mfa_password(vt)

    email_code = input("Email verification code: ")

    jackpot = ce.finalize_auth(vtt, email_code)


In [4]:
import json

critical_info = ce.get_august_critical_info(jackpot["data"]["verifyLoginCode"]["session"]["chirpAccessToken"], jackpot["data"]["verifyLoginCode"]["session"]["augustAccessToken"])
#print(critical_info)
august_info = ce.get_august_details(critical_info["augustAccessToken"], critical_info["augustApiKey"], critical_info["lockId"])

with open ('door_lock.json', 'w') as fp:
    json.dump(august_info, fp, indent=4)